Data kurs Japan Yen ke rupiah pada tahun 2019 dari monexnews.com/kurs-valuta-asing.htm?kurs=JPY

In [9]:
from flask import Flask, render_template
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import BytesIO
import base64
import matplotlib.pyplot as plt
import dateparser

In [10]:
app = Flask(__name__)

In [11]:
def scrap(url):
    url_get = requests.get('https://monexnews.com/kurs-valuta-asing.htm?kurs=JPY&searchdatefrom=01-01-2019&searchdateto=31-12-2019')
    soup = BeautifulSoup(url_get.content,"html.parser")
    
    table = soup.find('table', attrs={'class':'table'})
    tr = table.find_all('tr')
    
    temp = [] #initiating a tuple
    
    for i in range(1, len(tr)):
        row = table.find_all('tr')[i]
        #use the key to take information here
        #name_of_object = row.find_all(...)[0].text
        
        #get date
        tanggal = row.find_all('td')[0].text
        tanggal = tanggal.strip() #for removing the excess whitespace
        
        #get ask
        jual = row.find_all('td')[1].text
        jual = jual.strip() #for removing the excess whitespace
        
        #get bid
        beli = row.find_all('td')[2].text
        beli = beli.strip() #for removing the excess whitespace
        
        temp.append((tanggal,jual,beli)) #append the needed information
    
    temp2 = temp[::-1] #remove the header
    
    jpy = pd.DataFrame(temp2, columns = ('tanggal','jual','beli'))#creating the dataframe
    #data wranggling -  try to change the data type to right data type
    jpy['jual'] = jpy['jual'].str.replace(',', '.')
    jpy['beli'] = jpy['beli'].str.replace(',', '.')
    jpy[['jual','beli']] = jpy[['jual','beli']].astype('float64')
    jpy['tanggal'] = jpy['tanggal'].apply(lambda x: dateparser.parse(x))
    
    jpy['month']= jpy['tanggal'].dt.month_name()
    
    jpy[['month','jual','beli']].groupby(['month']).mean()

    #end of data wranggling

    return jpy

In [7]:
scrap('https://monexnews.com/kurs-valuta-asing.htm?kurs=JPY&searchdatefrom=01-01-2019&searchdateto=31-12-2019')

,tanggal,jual,beli,month
0,2019-01-02,132.89,131.54,January
1,2019-01-03,135.44,134.08,January
2,2019-01-04,133.51,132.17,January
3,2019-01-07,130.86,129.51,January
4,2019-01-08,129.84,128.52,January
...,...,...,...,...
241,2019-12-23,128.39,127.10,December
242,2019-12-26,128.27,126.98,December
243,2019-12-27,128.11,126.83,December
244,2019-12-30,128.30,127.02,December


In [12]:
@app.route("/")
def index():
    df = scrap('https://monexnews.com/kurs-valuta-asing.htm?kurs=JPY&searchdatefrom=01-01-2019&searchdateto=31-12-2019') #insert url here

    #This part for rendering matplotlib
    fig = plt.figure(figsize=(5,2),dpi=300)
    df.plot()
    
    #Do not change this part
    plt.savefig('plot1',bbox_inches="tight") 
    figfile = BytesIO()
    plt.savefig(figfile, format='png')
    figfile.seek(0)
    figdata_png = base64.b64encode(figfile.getvalue())
    result = str(figdata_png)[2:-1]
    #This part for rendering matplotlib

    #this is for rendering the table
    df = df.to_html(classes=["table table-bordered table-striped table-dark table-condensed"])

    return render_template("index.html", table=df, result=result)


if __name__ == "__main__": 
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2020-05-15 22:40:28,815] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\yohana\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\yohana\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\yohana\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\yohana\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\yohana\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\yohana\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-1

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x000001526E6EA1F8> (for post_execute):


ValueError: view limit minimum -36740.42249999999 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

<Figure size 1500x600 with 0 Axes>

ValueError: view limit minimum -36740.42249999999 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

<Figure size 432x288 with 1 Axes>